In [1]:
from function import *

2022-07-28 10:38:32.001411: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
print(os.getcwd())
os.chdir("/home/wmbio/WORK/gitworking/Dependency_prediction/")
print(os.getcwd())

/home/wmbio/WORK/gitworking/Dependency_prediction/prediction/src
/home/wmbio/WORK/gitworking/Dependency_prediction


In [16]:
with tf.device('/cpu:0'):
    model_name = "model_paper_exp_0727"
    model_saved = models.load_model("prediction/custom_model/%s.h5" % model_name, 
                                    custom_objects={"coeff_determination": coeff_determination})

    # model_name = "model_paper"
    # model_saved = models.load_model("prediction/model/%s.h5" % model_name)
# model_paper is the full 4-omics DeepDEP model used in the paper
# user can choose from single-omics, 2-omics, or full DeepDEP models from the
# /data/full_results_models_paper/models/ directory

ValueError: Unknown layer: Merge

* **Custom**

In [71]:
# load TCGA genomics data and gene fingerprints
data_mut, sample_names_mut, gene_names_mut = load_data(
    "preprocessing/DATA/2022-07-13/predict_mut_prediction.txt")
data_exp, sample_names_exp, gene_names_exp = load_data(
    "preprocessing/DATA/2022-07-13/predict_exp_prediction.txt")
data_cna, sample_names_cna, gene_names_cna = load_data(
    "preprocessing/DATA/2022-07-13/predict_cna_prediction.txt")
data_meth, sample_names_meth, gene_names_meth = load_data(
    "preprocessing/DATA/2022-07-13/predict_meth_prediction.txt")
data_fprint_DepOIs, gene_names_fprint, function_names_fprint = load_data(
    "preprocessing/DATA/2022-07-13/predict_fingerprint_prediction.txt")
print("\n\nDatasets successfully loaded.\n\n")



Datasets successfully loaded.




In [60]:
batch_size = 300
first_to_predict = 10
# predict the first 10 samples for DEMO ONLY, for all samples please substitute 10 by data_mut_tcga.shape[0]
# prediction results of all 8238 TCGA samples can be found in /data/full_results_models_paper/predictions/

In [ ]:
with tf.deshapee('/cpu:0'):
    t = time.time()
    data_pred = np.zeros((first_to_predict, data_fprint_DepOIs.shape[0]))
    for z in np.arange(0, first_to_predict):
        data_pred_tmp = model_saved.predict([
            data_mut[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_exp[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_cna[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_meth[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_fprint_DepOIs
        ], 
            batch_size=batch_size, verbose=0)
        data_pred[z] = np.transpose(data_pred_tmp)
        print("Unscreended sample %d predicted..." % z)

In [ ]:
# write prediction results to txt
data_pred_df = pd.DataFrame(data=np.transpose(data_pred), index=gene_names_fprint, columns=sample_names_mut[0:first_to_predict])

In [ ]:
data_pred_df.to_csv('custom_mut_exp.csv')

In [ ]:
# pd.DataFrame.to_csv(data_pred_df, path_or_buf="/results/predictions/tcga_predicted_data_%s_demo.txt" % model_name, sep='\t', index_label='CRISPR_GENE', float_format='%.4f')
# print("\n\nPrediction completed in %.1f mins.\nResults saved in /results/predictions/tcga_predicted_data_%s_demo.txt\n\n" % (
#     (time.time()-t)/60, model_name))